In [1]:
import socketio
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.api.pset
import ifcopenshell.util.element
import pandas as pd

# Create a Socket.IO client
sio = socketio.Client()
d = {}

# Define event handlers
@sio.event
def connect():
    print("Connected to the server.")

@sio.event
def disconnect():
    print("Disconnected from the server.")

@sio.event
def user_opens(data):
    print(data)
    print(data['model_path'])
    d['model_path'] = data['model_path']
    d['ifc'] = ifcopenshell.open(data['model_path'])
    print('user_opens', d['ifc'])

def get_psets(e): return ifcopenshell.util.element.get_psets(e, psets_only=True)
def get_qtos(e): return ifcopenshell.util.element.get_psets(e, qtos_only=True)

@sio.on('highlighting')
def on_event(data):
    print("Data received:", data)
    d['guids'] = data['guids']
    ifc = d['ifc']
    for guid in d['guids']:
        e = ifc.by_guid(guid)
        psets = get_psets(e)
        for key, pset in dict(sorted(psets.items())).items():
            if "Common" in key:
                print(key, pd.DataFrame.from_dict(pset, 'index'))
        qtos = get_qtos(e)
        for k, qto in dict(sorted(qtos.items())).items():
            if "Qto" in k:
                print(k, pd.DataFrame.from_dict(qto, 'index'))

# App's public URL on Google Cloud
cloud_run_url = 'https://pyijs-196533177105.europe-west6.run.app/' 

# Connect to the Flask-SocketIO server
sio.connect(cloud_run_url)
sio.emit('feeds', 'Hello from Jupyter coding frontend!')

# Keep the client running to listen for events
sio.wait()

Note: API not available due to missing dependencies: geometry.add_representation - No module named 'bpy'
Note: API not available due to missing dependencies: grid.create_axis_curve - No module named 'bpy'
Connected to the server.
{'model_path': './static/ifc/revicasa/corrkw/abb.ifc'}
./static/ifc/revicasa/corrkw/abb.ifc
user_opens <ifcopenshell.file.file object at 0x000001F56DD4A2D0>
Data received: {'guids': ['1PrNcvG3L0lAOCHYjoFAUL']}
Pset_WallCommon                                             0
IsExternal                               True
ThermalTransmittance                     0.35
id                                      16860
Reference             wall_exterior_WDH_10.00
LoadBearing                             False
ExtendToStructure                       False
Qto_WallBaseQuantities                                0
NetVolume               3.430616
GrossFootprintArea      1.376999
Height                  2.760000
Width                   0.100000
GrossVolume             3.800516
L

Disconnected from the server.
Connected to the server.
Disconnected from the server.
Connected to the server.
Disconnected from the server.
Connected to the server.
Disconnected from the server.
Connected to the server.
Disconnected from the server.
Connected to the server.
Disconnected from the server.
Connected to the server.
Disconnected from the server.
Connected to the server.
Disconnected from the server.
Connected to the server.
Disconnected from the server.
